In [ ]:
pip install pydicom

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/MyDrive/MiCM 2021- Deep Learning to Predict Disease Progression in Patients with Polycystic Kidney Disease/dataloader_data_kaggle/dicom

/content/drive/MyDrive/MiCM 2021- Deep Learning to Predict Disease Progression in Patients with Polycystic Kidney Disease/dataloader_data_kaggle/dicom


In [ ]:
import os
import pandas as pd
import pydicom

In [ ]:
pwd

'/content/drive/My Drive/MiCM 2021- Deep Learning to Predict Disease Progression in Patients with Polycystic Kidney Disease/dataloader_data_kaggle/dicom'

In [ ]:
ls

1/  10/  11/  2/  3/  4/  5/  6/  8/  9/


In [ ]:
data_dir = '/content/drive/My Drive/MiCM 2021- Deep Learning to Predict Disease Progression in Patients with Polycystic Kidney Disease/dataloader_data_kaggle/dicom/'
patients = os.listdir(data_dir)

In [ ]:
import numpy as np

In [ ]:
for patient in patients:
  path = data_dir + patient
  slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
  slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
  print(slices[0].pixel_array.shape, len(slices))
  
  try:
      slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
  except:
      slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
  print('---------------')
  print('slice thickness for patient %s: %d', patient, slice_thickness)
  print('---------------')
  

(256, 256) 53
---------------
slice thickness for patient %s: %d 4 3.0
---------------
(256, 256) 56
---------------
slice thickness for patient %s: %d 9 3.0
---------------
(256, 256) 52
---------------
slice thickness for patient %s: %d 5 3.0
---------------
(256, 256) 54
---------------
slice thickness for patient %s: %d 11 3.0
---------------
(256, 256) 53
---------------
slice thickness for patient %s: %d 3 3.0
---------------
(256, 256) 48
---------------
slice thickness for patient %s: %d 10 3.0
---------------
(256, 256) 54
---------------
slice thickness for patient %s: %d 6 3.0
---------------
(256, 256) 55
---------------
slice thickness for patient %s: %d 1 3.0
---------------
(256, 256) 57
---------------
slice thickness for patient %s: %d 8 3.0
---------------
(256, 256) 52
---------------
slice thickness for patient %s: %d 2 3.0
---------------


In [ ]:
len(slices)

52

In [ ]:
print(slices[5])

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 192
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.9328.50.51.247507555555569480377095724517065935886
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.27'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'OTHER']
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.9328.50.51.247507555555569480377095724517065935886
(0008, 0020) Study Date                          DA: '19901119'
(0008, 0023) Content Date             

We see that the images are not of the same size. We need to fix that because our neural network would need data with same dimensions as input. This can be done by resampling the full dataset to a certain isotropic resolution. 

Question: When do we need to do the segmentation? After?